In [ ]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.model_selection

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv("wine.data.csv")

target = data['label']
data = data.drop(['label'], axis=1)
data = data.to_numpy()
target = target.to_numpy()

In [ ]:
def train(X_train, y_train):
    svm = LinearSVC()
    svm.fit(X_train, y_train)
    
    mlp = MLPClassifier()
    mlp.fit(X_train, y_train)
    
    return svm, mlp

In [ ]:
def predict(X_valid, svm, mlp):

    pred_mlp = mlp.predict(X_valid)
    pred_svm = svm.predict(X_valid)
    features = np.column_stack((pred_svm, pred_mlp))
    
    return features

In [ ]:
def meta_learn(features, y_valid):
    
    logistic = LogisticRegression()
    logistic.fit(features, y_valid)
    
    return logistic

In [ ]:
def ensemble_predict(X_test, y_test, svm, mlp, logistic):

    svm_preds = predict(X_test, svm, mlp)
    
    meta_preds = logistic.predict(svm_preds)
    
    svm_acc = accuracy_score(y_test, svm_preds[:, 0])
    mlp_acc = accuracy_score(y_test, svm_preds[:, 1]) 
    logistic_acc = accuracy_score(y_test, meta_preds)
    
    return svm_acc, mlp_acc, logistic_acc

In [ ]:
performances = []

kf = sklearn.model_selection.KFold(10, shuffle=False)
for rest_index, test_index in kf.split(data):
    
    ## Step 1. Train/test split.
    midpoint = int(len(rest_index)/2)
    
    X_train = data[rest_index[0:midpoint]]
    y_train = target[rest_index[0:midpoint]]

    X_valid = data[rest_index[midpoint:]]
    y_valid = target[rest_index[midpoint:]]
    
    X_test = data[test_index]
    y_test = target[test_index]

    ## Step 2
    svm, mlp = train(X_train, y_train)

    ## Step 3
    features = predict(X_valid, svm, mlp)
    
    ## Step 4
    logistic = meta_learn(features, y_valid)

    ## Step 5
    accuracies = ensemble_predict(X_test, y_test, svm, mlp, logistic)

    performances.append(accuracies)

avg_accs = np.array(performances).mean(axis=0)
print(avg_accs)